In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from transformers import BertTokenizer,BertForSequenceClassification
from sklearn.model_selection import train_test_split



/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv('/kaggle/input/airline-reviews/BA_AirlineReviews.csv')
df.head()

,Unnamed: 0,OverallRating,ReviewHeader,Name,Datetime,VerifiedReview,ReviewBody,TypeOfTraveller,SeatType,Route,DateFlown,SeatComfort,CabinStaffService,GroundService,ValueForMoney,Recommended,Aircraft,Food&Beverages,InflightEntertainment,Wifi&Connectivity
0,0,1.0,"""Service level far worse then Ryanair""",L Keele,19th November 2023,True,4 Hours before takeoff we received a Mail stat...,Couple Leisure,Economy Class,London to Stuttgart,November 2023,1.0,1.0,1.0,1.0,no,NaN,NaN,NaN,NaN
1,1,3.0,"""do not upgrade members based on status""",Austin Jones,19th November 2023,True,I recently had a delay on British Airways from...,Business,Economy Class,Brussels to London,November 2023,2.0,3.0,1.0,2.0,no,A320,1.0,2.0,2.0
2,2,8.0,"""Flight was smooth and quick""",M A Collie,16th November 2023,False,"Boarded on time, but it took ages to get to th...",Couple Leisure,Business Class,London Heathrow to Dublin,November 2023,3.0,3.0,4.0,3.0,yes,A320,4.0,NaN,NaN
3,3,1.0,"""Absolutely hopeless airline""",Nigel Dean,16th November 2023,True,"5 days before the flight, we were advised by B...",Couple Leisure,Economy Class,London to Dublin,December 2022,3.0,3.0,1.0,1.0,no,NaN,NaN,NaN,NaN
4,4,1.0,"""Customer Service is non existent""",Gaylynne Simpson,14th November 2023,False,"We traveled to Lisbon for our dream vacation, ...",Couple Leisure,Economy Class,London to Lisbon,November 2023,1.0,1.0,1.0,1.0,no,NaN,1.0,1.0,1.0


In [3]:
df.shape

(3701, 20)

In [4]:
df.isnull().sum()

Unnamed: 0                  0
OverallRating               5
ReviewHeader                0
Name                        0
Datetime                    0
VerifiedReview              0
ReviewBody                  0
TypeOfTraveller           771
SeatType                    2
Route                     775
DateFlown                 778
SeatComfort               116
CabinStaffService         127
GroundService             846
ValueForMoney               1
Recommended                 0
Aircraft                 1779
Food&Beverages            386
InflightEntertainment    1150
Wifi&Connectivity        3092
dtype: int64

In [5]:
columns_to_extract = ['ReviewBody', 'OverallRating']

# Create a new DataFrame with only the selected columns
d1 = df[columns_to_extract].copy()

# Display the new DataFrame
d1.head()

,ReviewBody,OverallRating
0,4 Hours before takeoff we received a Mail stat...,1.0
1,I recently had a delay on British Airways from...,3.0
2,"Boarded on time, but it took ages to get to th...",8.0
3,"5 days before the flight, we were advised by B...",1.0
4,"We traveled to Lisbon for our dream vacation, ...",1.0


In [6]:
df['OverallRating'].nunique

<bound method IndexOpsMixin.nunique of 0       1.0
1       3.0
2       8.0
3       1.0
4       1.0
       ... 
3696    1.0
3697    9.0
3698    5.0
3699    4.0
3700    4.0
Name: OverallRating, Length: 3701, dtype: float64>

In [7]:
d1.shape

(3701, 2)

In [8]:
d1=d1.dropna()

In [9]:
d1.shape

(3696, 2)

In [10]:
d1['ReviewBody'].isnull().sum()

0

In [11]:
d1['OverallRating'].isnull().sum()

0

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
d1['r'] = le.fit_transform(d1['OverallRating'])

In [13]:
d1['r'].unique()

array([0, 2, 7, 6, 1, 5, 3, 4, 9, 8])

In [14]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=11)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
X = list(d1["ReviewBody"])
y = list(d1["r"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [16]:
model = model.to('cuda')

In [17]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [18]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [24]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [19]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [26]:
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=10,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    
)

In [27]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [28]:
trainer.evaluate()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 2.497309446334839,
 'eval_runtime': 12.6685,
 'eval_samples_per_second': 58.412,
 'eval_steps_per_second': 7.341}

In [24]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.677000
1000,1.302000
1500,0.982600
2000,0.696400
2500,0.501000
3000,0.296000
3500,0.152400


TrainOutput(global_step=3700, training_loss=0.7629597638104413, metrics={'train_runtime': 1658.0267, 'train_samples_per_second': 17.828, 'train_steps_per_second': 2.232, 'total_flos': 7778191280824320.0, 'train_loss': 0.7629597638104413, 'epoch': 10.0})

In [25]:
df.head(500)

,Unnamed: 0,OverallRating,ReviewHeader,Name,Datetime,VerifiedReview,ReviewBody,TypeOfTraveller,SeatType,Route,DateFlown,SeatComfort,CabinStaffService,GroundService,ValueForMoney,Recommended,Aircraft,Food&Beverages,InflightEntertainment,Wifi&Connectivity
0,0,1.0,"""Service level far worse then Ryanair""",L Keele,19th November 2023,True,4 Hours before takeoff we received a Mail stat...,Couple Leisure,Economy Class,London to Stuttgart,November 2023,1.0,1.0,1.0,1.0,no,NaN,NaN,NaN,NaN
1,1,3.0,"""do not upgrade members based on status""",Austin Jones,19th November 2023,True,I recently had a delay on British Airways from...,Business,Economy Class,Brussels to London,November 2023,2.0,3.0,1.0,2.0,no,A320,1.0,2.0,2.0
2,2,8.0,"""Flight was smooth and quick""",M A Collie,16th November 2023,False,"Boarded on time, but it took ages to get to th...",Couple Leisure,Business Class,London Heathrow to Dublin,November 2023,3.0,3.0,4.0,3.0,yes,A320,4.0,NaN,NaN
3,3,1.0,"""Absolutely hopeless airline""",Nigel Dean,16th November 2023,True,"5 days before the flight, we were advised by B...",Couple Leisure,Economy Class,London to Dublin,December 2022,3.0,3.0,1.0,1.0,no,NaN,NaN,NaN,NaN
4,4,1.0,"""Customer Service is non existent""",Gaylynne Simpson,14th November 2023,False,"We traveled to Lisbon for our dream vacation, ...",Couple Leisure,Economy Class,London to Lisbon,November 2023,1.0,1.0,1.0,1.0,no,NaN,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,10.0,"""Good professional crew""",J Richards,6th November 2021,True,Another BA shuttle flight and a good one. Fant...,Business,Economy Class,Belfast to London,November 2021,4.0,5.0,4.0,4.0,yes,A321,4.0,NaN,NaN
496,496,6.0,"""Not dreadful but something for BA to think ab...",E Smyth,31st October 2021,True,Flight from Larnaca to LHR. Check in was ok. T...,Family Leisure,Business Class,Larnaca to London Heathrow,October 2021,2.0,4.0,3.0,3.0,yes,A321,2.0,NaN,3.0
497,497,10.0,"""could not have asked for better service""",Paul Frankelh,29th October 2021,True,Upon checking in at the Club desk in Lisbon ai...,Couple Leisure,Business Class,Lisbon to London Heathrow,September 2021,4.0,4.0,5.0,4.0,yes,A320,4.0,NaN,NaN
498,498,9.0,"""BA at its best""",N Cooper,29th October 2021,True,Club Europe on British Airways. Outstanding an...,Couple Leisure,Premium Economy,Madrid to London,October 2021,4.0,5.0,3.0,5.0,yes,Boeing 787,5.0,4.0,NaN


In [56]:
text1 = df['ReviewBody'][499]

In [57]:
text1='Good flight but boarding was chaos, was gifted with the beautiful A10 gate at LHR T5(, so we were bussed to the plane (first time I\'ve used a bus gate there). The A320 was at T3 at a gate with a jetway, nevertheless 168 passengers (full flight) boarded via steps - this caused an hour waiting at the "gate" as well as the baggage taking ages. Also the seat recline on G-GATL (9f) is broken.'

'Good flight but boarding was chaos, was gifted with the beautiful A10 gate at LHR T5(, so we were bussed to the plane (first time I\'ve used a bus gate there). The A320 was at T3 at a gate with a jetway, nevertheless 168 passengers (full flight) boarded via steps - this caused an hour waiting at the "gate" as well as the baggage taking ages. Also the seat recline on G-GATL (9f) is broken.'

In [58]:
label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Mapping:", label_mapping)

Label Mapping: {1.0: 0, 2.0: 1, 3.0: 2, 4.0: 3, 5.0: 4, 6.0: 5, 7.0: 6, 8.0: 7, 9.0: 8, 10.0: 9}


In [59]:
values_array = np.arange(1, 11)

In [60]:
text = text1
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions
     

SequenceClassifierOutput(loss=None, logits=tensor([[-0.8062, -0.9692, -2.0826, -1.6884, -0.4192,  3.2594,  6.5037,  1.3118,
         -2.8596, -1.1663, -0.7773]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[6.3793e-04, 5.4201e-04, 1.7802e-04, 2.6404e-04, 9.3936e-04, 3.7191e-02,
         9.5376e-01, 5.3041e-03, 8.1844e-05, 4.4503e-04, 6.5665e-04]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)


array([[6.3793344e-04, 5.4200669e-04, 1.7801515e-04, 2.6403720e-04,
        9.3936233e-04, 3.7190694e-02, 9.5376033e-01, 5.3041484e-03,
        8.1844075e-05, 4.4502775e-04, 6.5664668e-04]], dtype=float32)

In [61]:
ans = np.argmax(predictions)

In [62]:
ans

6

In [63]:
values_array[ans]

7

In [64]:
text1 = df['ReviewBody'][497]
text = text1
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions
ans = np.argmax(predictions)
a=values_array[ans]
print(a)
     

SequenceClassifierOutput(loss=None, logits=tensor([[-1.0268, -1.1284, -0.6248, -0.8242, -0.3447,  0.4248, -0.4512, -1.8940,
          0.0185,  6.1592, -0.2271]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[7.4687e-04, 6.7470e-04, 1.1164e-03, 9.1461e-04, 1.4773e-03, 3.1889e-03,
         1.3280e-03, 3.1376e-04, 2.1243e-03, 9.8645e-01, 1.6616e-03]],
       device='cuda:0', grad_fn=<SoftmaxBackward0>)
10


In [65]:
trainer.save_model('/kaggle/working/')

In [66]:
import shutil

# Replace 'your_archive_name.zip' with the desired name for your zip file
zip_filename = '/kaggle/working/airline.zip'

# Replace '/kaggle/working/' with the path of the directory you want to zip
directory_to_zip = '/kaggle/working/'

# Create a zip file
shutil.make_archive(zip_filename, 'zip', directory_to_zip)

print(f"Directory '{directory_to_zip}' zipped to '{zip_filename}'")

Directory '/kaggle/working/' zipped to '/kaggle/working/airline.zip'
